In [2]:
import numpy as np


In [3]:
import json

def read_jsonl(file_path):
    all_data = []
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            # print(data)  # Process the JSON object as needed
            all_data.append(data)
    return all_data

# Example usage
filedata = read_jsonl('./data/ReviewCritique.jsonl')

In [4]:
len(filedata)

100

In [5]:
filedata['decision']

TypeError: list indices must be integers or slices, not str

In [6]:
import pandas as pd

# Read the JSONL file into a DataFrame
df = pd.read_json('./data/ReviewCritique.jsonl', lines=True)

# Access the 'decision' column
decision_column = df['decision']

# Display the DataFrame or the specific column
print(decision_column)

0     accepted
1     rejected
2     rejected
3     rejected
4     rejected
        ...   
95    accepted
96    accepted
97    rejected
98    rejected
99    accepted
Name: decision, Length: 100, dtype: object


In [7]:
df['review#1'][0].keys()

dict_keys(['review', 'score'])

In [12]:
entiretext = list(map(lambda x: x['segment_text'], df['review#1'][0]['review']))

In [16]:
print('\n'.join(entiretext))

Summary Of The Paper:
The authors present a new "meta" prompting method to achieve better accuracy in few-shot prompting.
This is based on the observation that certain types of questions tend to be more amenable to few-shot prompting (specifically, QA-type prompts). This prompting method (AMA), includes:
Creating question() prompts, which transfer the original task into an open ended question
Creating answer() prompts, which transfer the result to the actual answer They then aggregate a range of answers for a given prompt into a final answer, and find a significant performance lift
Strength And Weaknesses:
Strengths:
This paper has impressive empirical results in that it up-levels the 6B parameter model to the accuracy of the 175B parameter model for most of the tasks
In general, the observation that question-based tasks are much more likely to be answered correctly than other formats of tasks is really interesting.
This is exactly the sort of quantitative/qualitative evaluation of ben

In [17]:
entiretext = list(map(lambda x: x['segment_text'], df['review#2'][0]['review']))
print('\n'.join(entiretext))

Summary Of The Paper:
This paper focuses on zero-shot setting and looks into generating prompts that will improve zero shot performance.
It has been known that LLMs behave very differently given even slightly different prompts, and a lot of effort can be spent finding the perfect prompt.
Authors propose to, given the input, generate several "imperfect" prompts for this input and then aggregate the results.
To generate the imperfect prompts, authors come up with a number of templates (question template and answer templates) that teach model to, given an input, generate the questions for the input and then to answer these questions. The final prediction is an aggregation of prediction from the answer step.
The aggregation happens via weak supervision (the model for this is also learnt).
The experiments show impressive improvements for smaller models, allowing them to match that of larger models few shot performance.
Additionally, authors provide some insights as to what makes the good pr